In [1]:
import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

tf.config.list_physical_devices('GPU')

#データ準備
path = "train-data/"

files = glob.glob(path + "train/*.jpg")   #.jpgのパスを取得
files = sorted(files)
df_label = pd.read_csv(path + "train.csv")

# file_pathとcsv行を抽出する
def inexclusion(num: int):
  return files[:num], df_label.head(num)

files, df_label = inexclusion(2500)

file_list = []
for file in files:
  file = cv2.imread(file)
  file_list.append(file)

#要素数の確認
print(len(file_list),len(df_label))

2023-12-25 03:04:42.189787: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 03:04:42.214616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 03:04:42.214630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 03:04:42.215221: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-25 03:04:42.219889: I tensorflow/core/platform/cpu_feature_guar

2500 2500


In [2]:
%%time

#画素値を正規化
file_list = [file.astype(float)/255 for file in file_list]
train_x, valid_x, train_y, valid_y = train_test_split(file_list, df_label, test_size=0.2)

# train_y, valid_y をダミー変数化
train_y = to_categorical(train_y["gender_status"])
valid_y = to_categorical(valid_y["gender_status"])

# リスト型を配列型に
train_x = np.array(train_x)
valid_x = np.array(valid_x)

CPU times: user 672 ms, sys: 1.4 s, total: 2.08 s
Wall time: 2.07 s


In [3]:
#層の定義
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

# モデルを構築
model.compile(optimizer=tf.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
%%time

# Early stoppingを適用してフィッティング
log = model.fit(train_x, train_y, epochs=100, batch_size=10, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                     min_delta=0, patience=10,
                                                         verbose=1)],
                validation_data=(valid_x, valid_y))

Epoch 1/100
200/200 [==============================] - 35s 172ms/step - loss: 3.8795 - accuracy: 0.2730 - val_loss: 1.6605 - val_accuracy: 0.2880
Epoch 2/100
200/200 [==============================] - 32s 158ms/step - loss: 1.6141 - accuracy: 0.2800 - val_loss: 1.6559 - val_accuracy: 0.2680
Epoch 3/100
200/200 [==============================] - 31s 156ms/step - loss: 1.6135 - accuracy: 0.2885 - val_loss: 1.6543 - val_accuracy: 0.2880
Epoch 4/100
200/200 [==============================] - 31s 156ms/step - loss: 1.6145 - accuracy: 0.2755 - val_loss: 1.6555 - val_accuracy: 0.2680
Epoch 5/100
200/200 [==============================] - 31s 157ms/step - loss: 1.6142 - accuracy: 0.2925 - val_loss: 1.6539 - val_accuracy: 0.2880
Epoch 6/100
200/200 [==============================] - 31s 156ms/step - loss: 1.6130 - accuracy: 0.3030 - val_loss: 1.6578 - val_accuracy: 0.2680
Epoch 7/100
200/200 [==============================] - 31s 156ms/step - loss: 1.6157 - accuracy: 0.3060 - val_loss: 1.6570 -

In [5]:
loss,accuracy = model.evaluate(valid_x,valid_y,verbose = 0)
print('Accuracy','{:.2f}'.format(accuracy))

Accuracy 0.29
